In [1]:
import torch 
import torch.nn as nn
# import fasttext as ft
import math
import bcolz
import numpy as np
import pickle

In [2]:
english_words = pickle.load(open(f'english_words.pkl', 'rb'))

Finding English Non Violating Sentences

In [ ]:
def violates(line):
    linelist = line.split()
    for word in linelist:
        if word.lower() not in english_words:
            return True
    return False

In [ ]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic

In [ ]:
indices = []
ind = 0
count = 1
englishSens = []
with open('train.en','r') as f:
    for line in f:
        if ind >923:
            if not violates(line):
                indices.append(ind)
                englishSens.append(line)
                count+=1
        
        ind+=1
        if count > 100:
            break

In [ ]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic.append(max(indices))
pickle.dump(lic, open(f'lastindexchecked.pkl', 'wb'))

In [ ]:
lic = pickle.load(open(f'lastindexchecked.pkl', 'rb'))
lic

Cross Checking Violation

In [ ]:
for i in englishSens:
    l = i.split()
    for j in l:
        if j.lower() not in english_words:
            print(j)

Calling Google Translate API to get German translations

In [ ]:
import os
from google.cloud import translate

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/shouvik/Desktop/DeepLearning/miniproject/transformergooglecloudkey.json"

tc = translate.TranslationServiceClient()

parent = tc.location_path('alphacreate-1123', 'global')

target = 'de'

output = tc.translate_text(
    parent = parent,
    contents = englishSens,
    target_language_code=target,
    
)

# print(output)

In [ ]:
germanSens = []
for translation in output.translations:
#         print(u"Translated text: {}".format(translation.translated_text))
    germanSens.append(translation.translated_text)

In [ ]:
pickle.dump(englishSens, open(f'subsampledEnglishSens6.pkl', 'wb'))
pickle.dump(germanSens, open(f'subsampledGermanSens6.pkl', 'wb'))

Checking Violation in GermanSens

In [ ]:
germanSens = pickle.load(open(f'subsampledGermanSens6.pkl', 'rb'))

In [ ]:
german_words = pickle.load(open(f'german_words.pkl', 'rb'))

In [ ]:
def giveViolatingCount():
    a = []
    gindices = []
    gind = 0 # index
    counter = 0 # number of violating sentences
    for line in germanSens:
        linelist = line.split()
        flag = 'green'
        for word in linelist:
            if word.lower() not in german_words:
                a.append(gind)
                counter+=1
                flag = 'red'
                break
        if flag == 'green':
            gindices.append(gind)
        gind+=1
        if gind > max(indices):
            break

    print('Violating count:{}'.format(counter))

In [ ]:
german_words[:10]

In [ ]:
giveViolatingCount()

In [ ]:
# a = pickle.load(open(f'subsampledGermanSens.pkl', 'rb'))
# b = pickle.load(open(f'subsampledGermanSens2.pkl', 'rb'))
c = pickle.load(open(f'subsampledEnglishSens.pkl', 'rb'))
d = pickle.load(open(f'subsampledEnglishSens2.pkl', 'rb'))
e = pickle.load(open(f'subsampledEnglishSens3.pkl', 'rb'))
f = pickle.load(open(f'subsampledEnglishSens4.pkl', 'rb'))
g = pickle.load(open(f'subsampledEnglishSens5.pkl', 'rb'))
h = pickle.load(open(f'subsampledEnglishSens6.pkl', 'rb'))
numchars = 0

for i in range(100):
    numchars+=( len(c[i]) + len(d[i] )+ len(e[i]) + len(f[i]) + len(g[i]) + len(h[i]))

In [ ]:
print('Characters left: {}'.format(500000 - numchars))

In [ ]:
#469988
#456047
#428047
##
#423148

Verifying Translations

In [ ]:
englishSens[7]

In [ ]:
germanSens[7]

Bert embeddings

In [11]:
import pickle
import transformers
from bert_embedding import BertEmbedding
import torch

In [10]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# line = c[0]
# print('Sentence: {}'.format(line))
# print()
# tokens = tokenizer.tokenize(line) 
# print('Tokens: {}'.format(tokens))
# print()
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print('TokenIDs: {}'.format(token_ids))
    

In [ ]:
# bert_embedding = BertEmbedding(model='bert_12_768_12', dataset_name='book_corpus_wiki_de_uncased')
# result = bert_embedding(tokens)

In [8]:
# tokenizer_de = transformers.AutoTokenizer.from_pretrained("bert-base-german-cased")
# model_de = transformers.AutoModelWithLMHead.from_pretrained("bert-base-german-cased")

In [36]:
# tokenizer_de_uc = transformers.AutoTokenizer.from_pretrained("dbmdz/bert-base-german-uncased")
# model_de_uc = transformers.AutoModelWithLMHead.from_pretrained("dbmdz/bert-base-german-uncased")

In [9]:
germanSens = pickle.load(open(f'subsampledGermanSens.pkl', 'rb'))
germanSens[0]

'Eisenzement ist eine gebrauchsfertige Paste, die mit einem Spachtel oder Finger als Filet in die Formkanten (Ecken) der Stahlbarrenform gelegt wird.'

In [12]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForPreTraining.from_pretrained('bert-base-uncased')
input_ids = torch.tensor(tokenizer_de.encode(germanSens[0], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs_de = model_de(input_ids)
# prediction_scores, seq_relationship_scores = outputs[:2]

In [17]:
outputs_de[0].shape

torch.Size([1, 42, 30000])

In [18]:
englishSens = pickle.load(open(f'subsampledEnglishSens.pkl', 'rb'))
englishSens[0]

'iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .'

In [20]:
# tokenizer_en = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
# model_en = transformers.AutoModelWithLMHead.from_pretrained('bert-base-uncased')

# input_ids = torch.tensor(tokenizer_en.encode(englishSens[0], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# outputs_en = model_en(input_ids)

In [23]:
outputs_en[0].shape

torch.Size([1, 39, 30522])

Word_Embeddings Of Sentences

In [26]:
englishSens = []
sentence_count = 0
with open('train.en','r') as f:
    for line in f:
        englishSens.append(line)
        sentence_count+=1
        if sentence_count == 50:
            break
        

In [ ]:
tokenizer_en = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
model_en = transformers.AutoModelWithLMHead.from_pretrained('bert-base-uncased')

In [48]:
eng_word_embeddings = []
for i in englishSens:
    input_ids = torch.tensor(tokenizer_en.encode(i, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    outputs_en = model_en(input_ids)
    eng_word_embeddings.append(outputs_en[0])
    

In [56]:
len(eng_word_embeddings),eng_word_embeddings[0].shape

(50, torch.Size([1, 39, 30522]))

In [28]:
germanSens = []
sentence_count = 0
with open('train.de','r') as f:
    for line in f:
        germanSens.append(line)
        sentence_count+=1
        if sentence_count == 50:
            break
        

In [54]:
tokenizer_de_uc = transformers.AutoTokenizer.from_pretrained("dbmdz/bert-base-german-uncased")
model_de_uc = transformers.AutoModelWithLMHead.from_pretrained("dbmdz/bert-base-german-uncased")

In [55]:
de_word_embeddings = []
for i in germanSens:
    input_ids = torch.tensor(tokenizer_de_uc.encode(i, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    outputs_de = model_de_uc(input_ids)
    de_word_embeddings.append(outputs_de[0])

In [57]:
len(de_word_embeddings),de_word_embeddings[0].shape

(50, torch.Size([1, 55, 31102]))